In [1]:
import cx_Oracle
import hashlib
import getpass

dsn_tns = cx_Oracle.makedsn('kritserv', 1521, service_name='mumbaidb')

conn = cx_Oracle.connect(user="fa", password="mumbaidb", dsn=dsn_tns,encoding="UTF-8")
c = conn.cursor()

def login(UserName,pwd):
    pwd_new = pwd.encode()  
    pwd_hash = hashlib.sha256(pwd_new).hexdigest()
    
    sql = '''select count(*) from demopass where username = :EntName and password = :Entpwd'''
    c.execute(sql,EntName = UserName,Entpwd = pwd_hash)
    sucess = c.fetchall()
    
    if sucess[0][0] == 1:
        print("Login Sucessful\n\nSelection Operation\n1.Deposit \n2.Withdrawal \n3.Check Balance")
        trantype = int(input())
        if trantype == 1:
            Balance=input("Enter Deposit Amount :")
            update(Name,Balance,"A")
            
        elif trantype == 2:
            Balance=input("Enter Withdrawal Amount :")
            update(Name,Balance,"W")
                
        elif trantype == 3:
            Balance=0
            update(Name,Balance,"B")
            
        return 1   

    else:
        print("Failed")
        return 0


def update(UserName,EntBalance,Flag):
    sql = '''select balance from demobank where username = :usrname'''
    c.execute(sql,usrname = UserName)
    userbal = c.fetchall()
    
    if Flag == "A":
        changebal = userbal[0][0] + int(EntBalance)
        sql = '''update demobank set balance = :entbal  where username = :usrname'''
        c.execute(sql,entbal = changebal,usrname = UserName)
        conn.commit()
        print('New Balance is ' + str(changebal))
        
    elif Flag == "W":
        if int(EntBalance) > int(userbal[0][0]):
            print("Cannot Withdraw More than Available Balance: " + str(userbal[0][0]))
            
        else:
            updatebal = userbal[0][0] - int(EntBalance)
            sql = '''update demobank set balance = :entbal  where username = :usrname'''
            c.execute(sql,entbal = updatebal,usrname = UserName)
            conn.commit()
            print('New Balance is ' + str(updatebal))

            
    elif Flag == "B":
        print("Balance is: " + str(userbal[0][0]))
    
def signup(UserName,pwd,Balance):
    pwd_new = pwd.encode()  
    pwd_hash = hashlib.sha256(pwd_new).hexdigest()
    
    sql = '''select count(*) from demopass where username = :EntName'''
    c.execute(sql,EntName = UserName)
    sucess = c.fetchall()
    
    if sucess[0][0] == 1:
        print("Login already exist")
        #break

    else:
        print("Inserting Record...")
        
        pwd_new = pwd.encode()  
        pwd_hash = hashlib.sha256(pwd_new).hexdigest()
        sql = '''Insert into demopass(username,password) values (:usrname,:usrpwd)'''

        c.execute(sql,usrname = UserName, usrpwd = pwd_hash)

        sql = '''Insert into demobank(username,balance) values (:usrname,:bal)'''

        c.execute(sql,usrname = UserName, bal = Balance)
        conn.commit()
        print("Record Inserted...")

    
    
print("Select Operation:\n1.Login\n2.signup")
choice=int(input())
if choice==1:
    i=0
    while i < 3:
        Name=input("Enter User Name :")
        Password=getpass.getpass("Enter Password :")
        result=login(Name,Password)
        if result == 1:
            i = 3
            break

        else:
            if i == 2:
                print('Access denied. 3 attempts completed')

            else:
                print('Access denied. Try again.')

            i += 1
        

elif choice==2:
    Name=input("Enter User Name :")
    Password=getpass.getpass("Enter Password :")
    Balance=input("Enter Balance :")
    print(signup(Name,Password,Balance))
else:
    print("enter correct choice")

Select Operation:
1.Login
2.signup
1
Enter User Name :Kalpanaxyz
Enter Password :········
Login Sucessful

Selection Operation
1.Deposit 
2.Withdrawal 
3.Check Balance
3
Balance is: 400.0
